<h1>CS689A: Computational Linguistics for Indian Languages</h1>
<h2>Assignment-2</h2>
<h2>Allan Robey 22111007</h2>
<h3>Question-2-a</h3>

**Dependencies:** `numpy, conllu, datasets, transformers, seqeval`

### Creating train-validation-test splits
> Replace the file accordingly

**Note: Skip this section if files are already available in form of train-val-test splits**

In [86]:
from conllu import parse

data_file = open("hi_hdtb-ud-dev.conllu", "r", encoding="utf-8").read()
sentences = parse(data_file)
len(sentences)

1659

In [87]:
sentences = [sentence.serialize() for sentence in sentences]

In [88]:
from sklearn.model_selection import train_test_split

## Split size may be adjusted here
train, test = train_test_split(sentences,test_size=0.20)
train, val = train_test_split(train,test_size=0.10)

In [89]:
len(train),len(val),len(test)

(1194, 133, 332)

> Storing the outputs in `data/` folder

In [90]:
import os

## Create directory data/ if does not exist
if not os.path.exists('data'):
    os.makedirs('data')
    
for l,s in zip([train,val,test],['train','validation','test']):
    with open(f'data/{s}.conllu','w',encoding='utf-8') as fp:
        fp.write(''.join(l))

### Tokenization

The following section does tokenization for UPOS tagging task

> Loading data

In [91]:
from datasets import load_dataset

**Note: If the above convention for saving files was not followed, the file `ConlluIndic.py` needs to be editied so that `__TRAINING_FILE__` etc. point correctly**

In [92]:
dataset = load_dataset('ConlluIndic.py')
dataset

Found cached dataset conllu_indic (C:/Users/user/.cache/huggingface/datasets/conllu_indic/conlluindic/1.0.0/ae7d14539a4ea56cbe8ad3d5a402bf11da8a8a92f07652e6c6bd78f60f7e185c)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 1194
    })
    validation: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 133
    })
    test: Dataset({
        features: ['id', 'tokens', 'lemmas', 'upos_tags'],
        num_rows: 332
    })
})

> Tokenization

In [93]:
from transformers import AutoTokenizer

model_checkpoint = 'ai4bharat/indic-bert'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at C:\Users\user/.cache\huggingface\hub\models--ai4bharat--indic-bert\snapshots\4842dd258ecc0546f0d660b76a3b22a9c632f401\config.json
Model config AlbertConfig {
  "_name_or_path": "ai4bharat/indic-bert",
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.

In [94]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            #if label % 2 == 1:
            #    label += 1
            new_labels.append(label)

    return new_labels


In [95]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["upos_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [96]:
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Loading cached processed dataset at C:/Users/user/.cache/huggingface/datasets/conllu_indic/conlluindic/1.0.0/ae7d14539a4ea56cbe8ad3d5a402bf11da8a8a92f07652e6c6bd78f60f7e185c\cache-846c38901e19b314.arrow
Loading cached processed dataset at C:/Users/user/.cache/huggingface/datasets/conllu_indic/conlluindic/1.0.0/ae7d14539a4ea56cbe8ad3d5a402bf11da8a8a92f07652e6c6bd78f60f7e185c\cache-9f632a5208e65290.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [97]:
tokenized_datasets.save_to_disk('D:/data/tokenized_datasets/')

### Fine-tuning and Evaluation

In [98]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
from datasets import load_metric, load_from_disk, load_dataset
import numpy as np

> Loads the model, tokenizer, data collator, metric and tokenized datasets

**Note: Don't forget to edit the file paths if the previous conventions were not followed**

In [99]:
model_checkpoint = 'ai4bharat/indic-bert'


tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_datasets = load_from_disk('D:/data/tokenized_datasets/')



data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
metric = load_metric('seqeval')


dataset = load_dataset('ConlluIndic.py')

upos_feature = dataset['train'].features['upos_tags']
label_names = upos_feature.feature.names



id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}


model = AutoModelForTokenClassification.from_pretrained(model_checkpoint,id2label=id2label,label2id=label2id)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at C:\Users\user/.cache\huggingface\hub\models--ai4bharat--indic-bert\snapshots\4842dd258ecc0546f0d660b76a3b22a9c632f401\config.json
Model config AlbertConfig {
  "_name_or_path": "ai4bharat/indic-bert",
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.

  0%|          | 0/3 [00:00<?, ?it/s]

loading configuration file config.json from cache at C:\Users\user/.cache\huggingface\hub\models--ai4bharat--indic-bert\snapshots\4842dd258ecc0546f0d660b76a3b22a9c632f401\config.json
Model config AlbertConfig {
  "_name_or_path": "ai4bharat/indic-bert",
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "id2label": {
    "0": "ADJ",
    "1": "ADP",
    "10": "PRON",
    "11": "PROPN",
    "12": "PUNCT",
    "13": "SCONJ",
    "14": "SYM",
    "15": "VERB",
    "16": "X",
    "17": "_",
    "2": "ADV",
    "3": "AUX",
    "4": "CCONJ",
    "5": "DET",
    "6": "INTJ",
    "7": "NOUN",
    "8": "NUM",
    "9": "PART"
  },
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "label2id": {
    "ADJ": "0",
    "ADP": "1",
    "ADV": "2",
    "AUX": "3",
    

> Metric computation

In [100]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }


In [101]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1194
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 133
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 332
    })
})

> Defining the trainer

`fp16 = True` may be set for GPU

Similarly `batch_size` can be increased

In [107]:
batch_size = 32
args = TrainingArguments(
    output_dir=f"model/upos",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_strategy="epoch",
    learning_rate=4e-4,
    num_train_epochs=10,
    weight_decay=0.004,
    save_total_limit=1,
    #fp16= True,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    #eval_dataset=tokenized_datasets["validation"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


> Train the model

In [105]:
trainer.train()

C:\Users\user\anaconda3\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1194
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 380
  Number of trainable parameters = 32866834


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.678270,0.687408,0.729320,0.707744,0.776244
2,No log,0.491261,0.784104,0.796893,0.790447,0.842428
3,No log,0.438230,0.813272,0.818641,0.815947,0.859084
4,No log,0.401737,0.833207,0.853592,0.843276,0.875740
5,No log,0.382244,0.860202,0.857864,0.859032,0.890861
6,No log,0.419318,0.863531,0.869903,0.866705,0.892615
7,No log,0.380274,0.874758,0.878835,0.876792,0.900723
8,No log,0.421361,0.872504,0.882330,0.877389,0.901381
9,No log,0.432009,0.882148,0.886602,0.884370,0.905325
10,No log,0.429093,0.887941,0.889320,0.888630,0.907955


***** Running Evaluation *****
  Num examples = 133
  Batch size = 32
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: DET seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: NUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: VERB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\

Saving model checkpoint to model/upos\checkpoint-114
Configuration saved in model/upos\checkpoint-114\config.json
Model weights saved in model/upos\checkpoint-114\pytorch_model.bin
tokenizer config file saved in model/upos\checkpoint-114\tokenizer_config.json
Special tokens file saved in model/upos\checkpoint-114\special_tokens_map.json
Deleting older checkpoint [model\upos\checkpoint-76] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 133
  Batch size = 32
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: DET seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.wa

Saving model checkpoint to model/upos\checkpoint-228
Configuration saved in model/upos\checkpoint-228\config.json
Model weights saved in model/upos\checkpoint-228\pytorch_model.bin
tokenizer config file saved in model/upos\checkpoint-228\tokenizer_config.json
Special tokens file saved in model/upos\checkpoint-228\special_tokens_map.json
Deleting older checkpoint [model\upos\checkpoint-190] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 133
  Batch size = 32
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: DET seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.w

Saving model checkpoint to model/upos\checkpoint-342
Configuration saved in model/upos\checkpoint-342\config.json
Model weights saved in model/upos\checkpoint-342\pytorch_model.bin
tokenizer config file saved in model/upos\checkpoint-342\tokenizer_config.json
Special tokens file saved in model/upos\checkpoint-342\special_tokens_map.json
Deleting older checkpoint [model\upos\checkpoint-304] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 133
  Batch size = 32
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: DET seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.w

TrainOutput(global_step=380, training_loss=0.2645432723195929, metrics={'train_runtime': 5714.8017, 'train_samples_per_second': 2.089, 'train_steps_per_second': 0.066, 'total_flos': 39144345836976.0, 'train_loss': 0.2645432723195929, 'epoch': 10.0})

> Evaluate the model

**Note: Make sure evaluation results are reported on test dataset, for this training args should be changed**

In [106]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 133
  Batch size = 32


C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: DET seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ADP seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: NUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: VERB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\user\anaconda3\lib\site-packages\seqeval\metrics\sequence_labeli

{'eval_loss': 0.429092675447464,
 'eval_precision': 0.8879410624272974,
 'eval_recall': 0.8893203883495145,
 'eval_f1': 0.8886301901435778,
 'eval_accuracy': 0.9079552925706772,
 'eval_runtime': 22.3762,
 'eval_samples_per_second': 5.944,
 'eval_steps_per_second': 0.223,
 'epoch': 10.0}

In [108]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 332
  Batch size = 32


{'eval_loss': 0.44906193017959595,
 'eval_precision': 0.8805875848996999,
 'eval_recall': 0.8915720454181992,
 'eval_f1': 0.8860457724094087,
 'eval_accuracy': 0.9054126473740621,
 'eval_runtime': 44.0241,
 'eval_samples_per_second': 7.541,
 'eval_steps_per_second': 0.25}

# Question - 2b

## Precision, Recall , F-score for Validation dataset (Micro)

'precision': 0.8879410624272974,
'recall': 0.8893203883495145,
'f-score': 0.8886301901435778

## Precision, Recall , F-score for Test dataset (Micro)

'precision': 0.8805875848996999,
'recall': 0.8915720454181992,
'f-score': 0.8860457724094087


## Precision, Recall , F-score for dataset (Macro)

### Macro-average precision = (P1+P2)/2
### Macro-average recall = (R1+R2)/2 

### The Macro-average F-Score will be simply the harmonic mean of these two figures.



In [111]:
precision1 =  0.8879410624272974
recall1 = 0.8893203883495145
precision2 = 0.8805875848996999
recall2 = 0.8915720454181992 

macro_precision = (precision1 + precision2)/2
macro_recall = (recall1 + recall2)/2
macro_f_score = 2*(macro_precision * macro_recall) /(macro_precision + macro_recall)
print(" Macro-average Precision: ",macro_precision)
print(" Macro-average Recall: ",macro_recall)
print(" Macro-average F-score: ",macro_f_score)



 Macro-average Precision:  0.8842643236634986
 Macro-average Recall:  0.890446216883857
 Macro-average F-score:  0.887344503502727
